In [ ]:
import argparse
import requests
import urllib.parse
from urllib.parse import urlencode
import os
import zipfile

In [ ]:
def down(link, path):
    try:
        with open(path, "wb") as file:
            download_response = requests.get(link, stream=True)
            download_response.raise_for_status()  # Raise an error for bad status codes
            total_length = download_response.headers.get('content-length')

            if total_length is None:  # No content length header
                file.write(download_response.content)
            else:
                dl = 0
                total_length = int(total_length)
                for chunk in download_response.iter_content(chunk_size=1024):
                    if chunk:
                        dl += len(chunk)
                        file.write(chunk)
                        file.flush()
                        done = int(50 * dl / total_length)
                        os.sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )
                        os.sys.stdout.flush()
                print()  # New line after progress bar
    except requests.RequestException as e:
        print(f"Download failed: {e}")
        if os.path.exists(path):
            os.remove(path)  # Remove partial download

In [ ]:
def unzip_file(filepath):
    """
    Extracts a zip archive into the same directory, preserving the inside subdirectory structure,
    and removes the original zip file upon successful extraction.
    
    :param filepath: Path to the zip archive file.
    """
    if not os.path.isfile(filepath) or not filepath.lower().endswith('.zip'):
        raise ValueError("The provided filepath is not a valid zip file.")
    
    extract_to = os.path.dirname(filepath)
    
    with zipfile.ZipFile(filepath, 'r') as zip_ref:
        zip_ref.extractall(path=extract_to)
    
    os.remove(filepath)
    print(f"Successfully extracted {filepath} and removed the original file.")

In [ ]:
dataset_url = "https://disk.yandex.ru/d/mwyS4ZoZNixv5w"
url = f"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={dataset_url}"
response = requests.get(url)
download_url = response.json()["href"]
print(download_url)

In [ ]:
# down(download_url, "../dataset/dataset.zip")